In [2]:
# Deploying custom keras model on aws sagemaker

from keras.models import load_model
from sagemaker.tensorflow.model import TensorFlowModel
from keras import backend as K
import boto3
import tarfile
import config

from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants


Model_Location ='model.hdf5'
Model_S3_Location = '/model/model.tar.gz'


def get_role():
    iam = boto3.client('iam', aws_access_key_id = config.aws_access_key_id, aws_secret_access_key = config.aws_secret_access_key , region_name= config.region)
    role = iam.get_role(RoleName=config.sagemakerRoleName)['Role']['Arn']
    return role


def convert_to_aws(loaded_model):
    version = '1'
    directory = 'export/Servo/' + version
    build = builder.SavedModelBuilder(directory)
    
    signature = predict_signature_def(inputs={"inputs": loaded_model.input}, outputs={"score": loaded_model.output})

    with K.get_session() as sess:
        build.add_meta_graph_and_variables(sess=sess, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
        build.save()
    
    with tarfile.open('model.tar.gz', mode='w:gz') as archive:
        archive.add('export', recursive=True)


# load the keras model
model = load_model(Model_Location) 

# covert model into aws model format
convert_to_aws(model)                    

# upload into s3 bucket 
s3 = boto3.resource('s3', aws_access_key_id = config.aws_access_key_id, aws_secret_access_key = config.aws_secret_access_key , region_name= config.region)
s3.meta.client.upload_file('model.tar.gz', config.bucket, Model_S3_Location)

!touch train.py                    #create an empty python file for model creation

# create a tensorflow model 
model = TensorFlowModel(model_data = 's3://'+ config.bucket + Model_S3_Location,
                                  role = get_role(),
                                  framework_version = '1.15',
                                  entry_point = 'train.py')

# deploy the model
predictor = model.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge')


        

















Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: export/Servo/1/saved_model.pb


update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-------------!